## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,6/22/20,Wards Island,11.57,1.0,30.0,1.50,7.71,Specialized
1,7/8/20,From Citarella,1.18,NaN,13.0,0.22,5.45,Citibike
2,9/28/19,Down ES Up WS,16.16,1.0,35.0,1.58,10.21,Specialized
3,9/16/16,Citibike to Work Midday,1.09,NaN,13.0,0.22,5.03,Citibike
4,8/15/20,Wards Island,12.70,1.0,36.0,1.60,7.94,Specialized
...,...,...,...,...,...,...,...,...
348,8/4/19,Down ES Up WS,16.79,2.0,2.0,2.03,8.26,Specialized
349,9/4/16,Down ES UP WS,16.70,1.0,56.0,1.93,8.64,Specialized
350,8/31/19,Up UWS Down WS Up ES,21.56,2.0,25.0,2.42,8.92,Specialized
351,7/8/20,To Citarella,1.08,NaN,9.0,0.15,7.20,Citibike


In [34]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,6/22/20,Wards Island,11.57,1.0,30.0,1.50,7.71,Specialized
1,1,7/8/20,From Citarella,1.18,NaN,13.0,0.22,5.45,Citibike
2,2,9/28/19,Down ES Up WS,16.16,1.0,35.0,1.58,10.21,Specialized
3,3,9/16/16,Citibike to Work Midday,1.09,NaN,13.0,0.22,5.03,Citibike
4,4,8/15/20,Wards Island,12.70,1.0,36.0,1.60,7.94,Specialized
...,...,...,...,...,...,...,...,...,...
348,348,8/4/19,Down ES Up WS,16.79,2.0,2.0,2.03,8.26,Specialized
349,349,9/4/16,Down ES UP WS,16.70,1.0,56.0,1.93,8.64,Specialized
350,350,8/31/19,Up UWS Down WS Up ES,21.56,2.0,25.0,2.42,8.92,Specialized
351,351,7/8/20,To Citarella,1.08,NaN,9.0,0.15,7.20,Citibike


In [5]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [6]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,11.57,1.50,7.71,Specialized
1,1.18,0.22,5.45,Citibike
2,16.16,1.58,10.21,Specialized
3,1.09,0.22,5.03,Citibike
4,12.70,1.60,7.94,Specialized
...,...,...,...,...
348,16.79,2.03,8.26,Specialized
349,16.70,1.93,8.64,Specialized
350,21.56,2.42,8.92,Specialized
351,1.08,0.15,7.20,Citibike


In [7]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
#new2_df = all_df[['Miles', 'Duration', 'Speed', 'Type']]
#new2_df

In [8]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [9]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [10]:
# Checking training data shape
X_train.shape

(264, 3)

In [11]:
# Checking testing data shape
X_test.shape

(89, 3)

In [12]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [13]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [14]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [15]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9550561797752809


In [16]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Citibike
2,Specialized
3,Citibike
4,Specialized
...,...
84,Citibike
85,Citibike
86,Specialized
87,Citibike


In [17]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
232,Specialized
299,Citibike
114,Specialized
141,Specialized
98,Specialized
...,...
313,Citibike
215,Citibike
64,Specialized
161,Citibike


In [18]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,232,Specialized
1,299,Citibike
2,114,Specialized
3,141,Specialized
4,98,Specialized
...,...,...
84,313,Citibike
85,215,Citibike
86,64,Specialized
87,161,Citibike


### Combining the test and prediction dataframes for comparison

In [19]:
# Combining the new test and predition dataframes horizontally
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,232,Specialized,Specialized
1,299,Citibike,Citibike
2,114,Specialized,Specialized
3,141,Specialized,Citibike
4,98,Specialized,Specialized
...,...,...,...
84,313,Citibike,Citibike
85,215,Citibike,Citibike
86,64,Specialized,Specialized
87,161,Citibike,Citibike


In [20]:
# Exporting Combined Test and Prediction Dataframe for examination
test_pred_file = "test_pred.csv"
test_pred_df.to_csv(os.path.join(data_path, test_pred_file))

In [22]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
3,141,Specialized,Citibike
48,290,Specialized,Citibike
59,179,Citibike,Specialized
67,120,Specialized,Citibike


In [39]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,141,Specialized,Citibike,7/15/19,Up Down ES,3.74,NaN,33.0,0.55,6.80,Specialized
1,290,Specialized,Citibike,6/28/20,Uptown Trio,3.95,NaN,26.0,0.43,9.12,Specialized
2,179,Citibike,Specialized,6/17/18,Work to Village,4.69,NaN,40.0,0.67,7.04,Citibike
3,120,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,NaN,39.0,0.65,6.54,Specialized


In [38]:
# Cleaning up err_details_df
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'})
err_details_df

KeyError: "['Type_y'] not found in axis"